In [ ]:
#%pip install ISLP

In [159]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from ISLP import load_data, confusion_table
from ISLP.models import ModelSpec as MS
import pandas as pd


from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text)
from sklearn.metrics import (accuracy_score,
                             log_loss)
from sklearn.ensemble import \
     (RandomForestRegressor as RF,
      GradientBoostingRegressor as GBR)
from ISLP.bart import BART

In [160]:
# load the training data train.csv
train_data = pd.read_csv('train.csv')
train_data.head()
print(train_data.shape)
print(train_data.columns)
# Remove some features that are supposed to be not useful for prediction

# Access the 'Name' column data
name_column = train_data['Name']

# check how many different names are there in the name column
name_list = name_column.unique().tolist()
print(f"Number of unique names: {len(name_list)}") #207 => too much, ignore name feature

# drop the Name feature
train_data = train_data.drop(columns=['Name'])
print(train_data.shape)

# check how many different locations are there in the Location column
location_column = train_data['Location']
location_list = location_column.unique().tolist()
print(location_list)
print(f"Number of unique locations: {len(location_list)}") 
train_data = train_data.drop(columns=['Location'])# =12, also drop location feature
print(train_data.shape)

# Drop the feature "New-Price" because there is not many data
train_data = train_data.drop(columns=['New_Price'])
print(train_data.shape) #1491, 12 -> cannot drop more

model = MS(train_data.columns, intercept=False)
D = model.fit_transform(train_data)

def replace_data(column, value):
    data = column
    for i in range(len(column)):
        try:
            data_value = float(column[i])
        except:
            data[i] = str(value)
            continue
        if data[i] == '\\N' or data[i] == '':
            data[i] = str(value)
            continue
    return data

def calculate_average (data):
    sum = 0
    count = 0
    for i in range(len(data)):
        if data[i] != '\\N' and data[i] != '':
            try:
                sum += float(data[i])
                count += 1
            except:
                continue
    average = sum / count
    return average
    

(4470, 16)
Index(['ID', 'Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type',
       'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Colour',
       'Seats', 'No. of Doors', 'New_Price', 'Price'],
      dtype='object')
Number of unique names: 207
(4470, 15)
['Coimbatore', 'Kochi', 'Hyderabad', 'Kolkata', 'Bangalore', 'Delhi', 'Pune', 'Chennai', 'Mumbai', 'Ahmedabad', 'Jaipur', '\\N']
Number of unique locations: 12
(4470, 14)
(4470, 13)


In [161]:
# preprocess the data
# calculate the average year
avg_year = int(calculate_average(D['Year']))
print("Average year:", avg_year)

# replace the missing year with the average year
D['Year'] = replace_data(D['Year'], avg_year)  

# convert D['Year'] to int
D['Year'] = pd.to_numeric(D['Year'])

#print(D['Year'])
print(D.shape)

Average year: 2013
(4470, 13)


/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)


In [162]:
# Preprocess the Kilometers_Driven data
average_km = int(calculate_average(D['Kilometers_Driven']))
print("Average Kilometers_Driven:", average_km)

# replace the missing km with the average km
D['Kilometers_Driven'] = replace_data(D['Kilometers_Driven'], average_km)
D['Kilometers_Driven'] = pd.to_numeric(D['Kilometers_Driven'])# covert to numeric
print(D.shape)

Average Kilometers_Driven: 59184
(4470, 13)


/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [163]:
# Fuel type preprocessing
fuel_types = D['Fuel_Type'].unique().tolist() # check how many different fuel types
print(fuel_types)
for fuel in fuel_types: # encode each fuel type into a separate binary feature
    D[f'Fuel_Type_{fuel}'] = D['Fuel_Type'].apply(lambda x: 1 if x == fuel else 0)
D = D.drop(columns=['Fuel_Type']) # drop the original Fuel_Type column
print(D.columns) # print to check
print(D.shape)

['Diesel', 'Petrol', 'CNG', 'LPG', 'Electric']
Index(['ID', 'Year', 'Kilometers_Driven', 'Transmission', 'Owner_Type',
       'Mileage', 'Engine', 'Power', 'Colour', 'Seats', 'No. of Doors',
       'Price', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol', 'Fuel_Type_CNG',
       'Fuel_Type_LPG', 'Fuel_Type_Electric'],
      dtype='object')
(4470, 17)


In [164]:
# Process the 'Transmission' column
print(D.shape)
transmission_types = D['Transmission'].unique().tolist() # check how many different transmission types
print(transmission_types)
for transmission in transmission_types: # encode each transmission type into a separate binary feature
    D[f'Transmission_{transmission}'] = D['Transmission'].apply(lambda x: 1 if x == transmission else 0)
D = D.drop(columns=['Transmission']) # drop the original Transmission column
print(D.columns) # print to check
print(D['Transmission_Manual']) # print to check
print(D.shape)

(4470, 17)
['Manual', 'Automatic', '\\N']
Index(['ID', 'Year', 'Kilometers_Driven', 'Owner_Type', 'Mileage', 'Engine',
       'Power', 'Colour', 'Seats', 'No. of Doors', 'Price', 'Fuel_Type_Diesel',
       'Fuel_Type_Petrol', 'Fuel_Type_CNG', 'Fuel_Type_LPG',
       'Fuel_Type_Electric', 'Transmission_Manual', 'Transmission_Automatic',
       'Transmission_\N'],
      dtype='object')
0       1
1       1
2       1
3       1
4       1
       ..
4465    1
4466    1
4467    0
4468    1
4469    0
Name: Transmission_Manual, Length: 4470, dtype: int64
(4470, 19)


In [165]:
#Process the 'Owner_Type' column
owner_types = D['Owner_Type'].unique().tolist() # check how many different owner types
print(owner_types)
for owner in owner_types: # encode each owner type into a separate binary feature
    D[f'Owner_Type_{owner}'] = D['Owner_Type'].apply(lambda x: 1 if x == owner else 0)
D = D.drop(columns=['Owner_Type']) # drop the original Owner_Type column
print(D.columns) # print to check
print(D.shape)


['First', 'Second', 'Third', '\\N', 'Fourth & Above']
Index(['ID', 'Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power',
       'Colour', 'Seats', 'No. of Doors', 'Price', 'Fuel_Type_Diesel',
       'Fuel_Type_Petrol', 'Fuel_Type_CNG', 'Fuel_Type_LPG',
       'Fuel_Type_Electric', 'Transmission_Manual', 'Transmission_Automatic',
       'Transmission_\N', 'Owner_Type_First', 'Owner_Type_Second',
       'Owner_Type_Third', 'Owner_Type_\N', 'Owner_Type_Fourth & Above'],
      dtype='object')
(4470, 23)


In [166]:
#Preprocess Mileage
print(D['Mileage'])
D['Mileage'] = D['Mileage'].str.split(' ').str[0] # extract the numeric part
mileage_average = calculate_average(D['Mileage'])
print("Average Mileage:", mileage_average)
# replace the missing mileage with the average mileage
D['Mileage'] = replace_data(D['Mileage'], mileage_average)
D['Mileage'] = pd.to_numeric(D['Mileage'])# covert to numeric
print(D.shape)

0        17.0 kmpl
1       21.43 kmpl
2        13.8 kmpl
3       21.27 kmpl
4        17.0 kmpl
           ...    
4465     16.0 kmpl
4466     27.3 kmpl
4467     12.7 kmpl
4468     24.7 kmpl
4469     12.0 kmpl
Name: Mileage, Length: 4470, dtype: object
Average Mileage: 18.211246642793192
(4470, 23)


/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)


In [167]:
#Process Engine
print(D['Engine'])
D['Engine'] = D['Engine'].str.split(' ').str[0] # extract the numeric part
# calculate the average engine power
average_engine_volumne = int(calculate_average(D['Engine']))
print("Average Engine Volume:", average_engine_volumne)
# replace the missing engine volume with the average engine volume
D['Engine'] = replace_data(D['Engine'], average_engine_volumne)
D['Engine'] = pd.to_numeric(D['Engine'])# covert to numeric
print(D.shape)

0       1405 CC
1       1364 CC
2       1299 CC
3       1396 CC
4       1497 CC
         ...   
4465    2179 CC
4466    1498 CC
4467    2179 CC
4468     796 CC
4469    2987 CC
Name: Engine, Length: 4470, dtype: object
Average Engine Volume: 1618
(4470, 23)


/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [168]:
# Process Power
print(D['Power'])
D['Power'] = D['Power'].str.split(' ').str[0] # extract the numeric part
# calculate the average power
average_power = calculate_average(D['Power'])
print("Average Power:", average_power)
# replace the missing power with the average power
D['Power'] = replace_data(D['Power'], average_power)
D['Power'] = pd.to_numeric(D['Power'])# covert to numeric
print(D.shape)

0          70 bhp
1        87.2 bhp
2          70 bhp
3       88.76 bhp
4         118 bhp
          ...    
4465      140 bhp
4466     98.6 bhp
4467    187.7 bhp
4468     47.3 bhp
4469      224 bhp
Name: Power, Length: 4470, dtype: object
Average Power: 113.15677360219904
(4470, 23)


/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [169]:
# Process Colour column
color_types = D['Colour'].unique().tolist() # check how many different color types
print(color_types)
for color in color_types: # encode each color type into a separate binary feature
    D[f'Colour_{color}'] = D['Colour'].apply(lambda x: 1 if x == color else 0)
D = D.drop(columns=['Colour']) # drop the original Colour column
print(D.columns) # print to check
print(D.shape)

['Others', 'Black/Silver', 'White', '\\N']
Index(['ID', 'Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power',
       'Seats', 'No. of Doors', 'Price', 'Fuel_Type_Diesel',
       'Fuel_Type_Petrol', 'Fuel_Type_CNG', 'Fuel_Type_LPG',
       'Fuel_Type_Electric', 'Transmission_Manual', 'Transmission_Automatic',
       'Transmission_\N', 'Owner_Type_First', 'Owner_Type_Second',
       'Owner_Type_Third', 'Owner_Type_\N', 'Owner_Type_Fourth & Above',
       'Colour_Others', 'Colour_Black/Silver', 'Colour_White', 'Colour_\N'],
      dtype='object')
(4470, 26)


In [170]:
# Process the Seats column
print(D['Seats'])
# calculate the average seats
average_seats = int(calculate_average(D['Seats']))
print("Average Seats:", average_seats)
# replace the missing seats with the average seats
D['Seats'] = replace_data(D['Seats'], average_seats)
D['Seats'] = pd.to_numeric(D['Seats'])# covert to numeric
print(D.shape)


0       5
1       5
2       5
3       5
4       5
       ..
4465    7
4466    5
4467    5
4468    5
4469    7
Name: Seats, Length: 4470, dtype: object
Average Seats: 5
(4470, 26)


/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)
/tmp/ipykernel_333811/1491415521.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = str(value)


In [ ]:
# Process No. of Doors column
#calculate the average no. of doors
average_doors = int(calculate_average(D['No. of Doors']))
print("Average No. of Doors:", average_doors)
# replace the missing no. of doors with the average no. of doors
D['No. of Doors'] = replace_data(D['No. of Doors'], average_doors)
D['No. of Doors'] = pd.to_numeric(D['No. of Doors'])# covert to numeric

print(D.shape)

In [ ]:
# last, we process the response variable Price
#calculate the average price
average_price = calculate_average(D['Price'])
print("Average Price:", average_price)
# replace the missing price with the average price
D['Price'] = replace_data(D['Price'], average_price)
D['Price'] = pd.to_numeric(D['Price'])# covert to numeric
print(D.shape)

In [ ]:
# we save processed D to a csv file
D.to_csv('train_processed.csv', index=False)
